In [120]:
pip install pandas sqlalchemy ipython-sql


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [121]:
pip install --upgrade prettytable

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [122]:
pip install --upgrade ipython-sql


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [123]:
import sqlite3
import pandas as pd


In [124]:
import prettytable
prettytable.__dict__['DEFAULT'] = prettytable.PLAIN_COLUMNS


In [125]:
# List of CSV file paths
csv_files = {
    "Appearances": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\appearances.csv",
    "Games": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\games.csv",
    "Leagues": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\leagues.csv",
    "Players": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\players.csv",
    "Shots": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\shots.csv",
    "Teams": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\teams.csv",
    "TeamStats": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\teamstats.csv",
}

In [ ]:
# Add these lines to enable SQLite extensions
def load_extension(conn):
    conn.enable_load_extension(True)
    try:
        # On Windows
        conn.load_extension('sqlite3_mod_spatialite')
    except:
        # On Unix/Linux/Mac
        conn.load_extension('mod_spatialite')
    conn.enable_load_extension(False)


In [126]:
# Create SQLite database (file-based)
db_file = "football_database.db"
conn = sqlite3.connect(db_file)  # Use a persistent database file


In [127]:
# Load each CSV into a separate table
for table_name, file_path in csv_files.items():
    try:
        # Specify a fallback encoding
        df = pd.read_csv(file_path, encoding='latin1')
        df.to_sql(table_name, conn, index=False, if_exists='replace')
        print(f"Loaded {file_path} into table {table_name}")
    except UnicodeDecodeError as e:
        print(f"Error loading {file_path}: {e}")


Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\appearances.csv into table Appearances
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\games.csv into table Games
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\leagues.csv into table Leagues
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\players.csv into table Players
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\shots.csv into table Shots
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\teams.csv into table Teams
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project -

In [128]:
# Verify tables are loaded
print("Tables in the database:")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

Tables in the database:
          name
0  Appearances
1        Games
2      Leagues
3      Players
4        Shots
5        Teams
6    TeamStats


In [129]:
# Load SQL magic
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [130]:
# Configure SQL magic display settings
%config SqlMagic.feedback = False
%config SqlMagic.autopandas = True  # This converts results to pandas DataFrames
%config SqlMagic.displaywidth = 1000  # Controls width

# Add pandas display options to control both width and length
pd.set_option('display.max_rows', 20)     # Limits number of rows
pd.set_option('display.max_columns', 10)  # Limits number of columns
pd.set_option('display.width', 1000)      # Controls overall display width
pd.set_option('display.max_colwidth', 100) # Controls maximum column width

c:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3284: UserWarning: Config option `displaywidth` not recognized by `SqlMagic`.  Did you mean one of: `displaycon, displaylimit`?
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [131]:
%sql sqlite:///football_database.db  


In [132]:
%%sql
SELECT COUNT(*)

FROM Leagues





 * sqlite:///football_database.db


,COUNT(*)
0,5


In [133]:

%%sql
--Get all teams IDs that played in the Premier League
SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1)
                  



 * sqlite:///football_database.db
(sqlite3.OperationalError) unrecognized token: "#"
[SQL: # Get all teams IDs that played in the Premier League
SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [134]:
%%sql
--Display every team name in the premier league
WITH premierLeagueIDs AS (SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1))

SELECT t.teamID, t.name
FROM teams t
WHERE t.teamID in (SELECT id
                   FROM premierLeagueIDs);



 * sqlite:///football_database.db


,teamID,name
0,71,Aston Villa
1,72,Everton
2,74,Southampton
3,75,Leicester
4,76,West Bromwich Albion
...,...,...
26,227,Cardiff
27,228,Fulham
28,229,Wolverhampton Wanderers
29,238,Sheffield United


In [135]:
%%sql
--Calculate how many goals Arsenal scored at home by season
SELECT 
    season, 
    SUM(homegoals) as Total_Home_Goals
        
FROM games g
WHERE g.homeTeamID = 83
GROUP BY season
ORDER BY season;

 * sqlite:///football_database.db


,season,Total_Home_Goals
0,2014,41
1,2015,31
2,2016,39
3,2017,54
4,2018,42
5,2019,36
6,2020,24


In [136]:
%%sql
#Calculate how many goals Arsenal scored at home vs away from the 2014 to 2020 season
SELECT 
    season, 
    SUM(CASE WHEN homeTeamID = 83 THEN homegoals ELSE 0 END) AS Total_Home_Goals,
    SUM(CASE WHEN awayTeamID = 83 THEN awaygoals ELSE 0 END) AS Total_Away_goals
FROM games
WHERE (homeTeamID = 83
    OR awayTeamID = 83)
GROUP BY season
ORDER BY season;

 * sqlite:///football_database.db
(sqlite3.OperationalError) near "#Calculate": syntax error
[SQL: #Calculate how many goals Arsenal scored at home vs away from the 2014 to 2020 season
SELECT 
    season, 
    SUM(CASE WHEN homeTeamID = 83 THEN homegoals ELSE 0 END) AS Total_Home_Goals,
    SUM(CASE WHEN awayTeamID = 83 THEN awaygoals ELSE 0 END) AS Total_Away_goals
FROM games
WHERE (homeTeamID = 83
    OR awayTeamID = 83)
GROUP BY season
ORDER BY season;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [138]:
%%sql
/* Shows all winners and losers in 2020 Premier League matches with: */
SELECT  g.season, 
        g.date, 
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.homeTeamID
        ) AS Home_Team,
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.awayTeamID
        ) AS Away_Team, 
        (CASE WHEN homeGoals > awayGoals THEN "Home Team Wins"
        WHEN homeGoals < awayGoals THEN "Away Team Wins"
        ELSE 'Tie' END) As Winning_Side
FROM games g
WHERE leagueID = 1 and season = '2020'
ORDER BY date DESC;



 * sqlite:///football_database.db


,season,date,Home_Team,Away_Team,Winning_Side
0,2020,2021-05-23 15:00:00,Arsenal,Brighton,Home Team Wins
1,2020,2021-05-23 15:00:00,Aston Villa,Chelsea,Home Team Wins
2,2020,2021-05-23 15:00:00,Fulham,Newcastle United,Away Team Wins
3,2020,2021-05-23 15:00:00,Leeds,West Bromwich Albion,Home Team Wins
4,2020,2021-05-23 15:00:00,Leicester,Tottenham,Away Team Wins
...,...,...,...,...,...
375,2020,2020-09-13 13:00:00,West Bromwich Albion,Leicester,Away Team Wins
376,2020,2020-09-12 19:00:00,West Ham,Newcastle United,Away Team Wins
377,2020,2020-09-12 16:30:00,Liverpool,Leeds,Home Team Wins
378,2020,2020-09-12 14:00:00,Crystal Palace,Southampton,Home Team Wins


In [139]:
%%sql
--Same query as above writen differently
SELECT g.season, 
    g.date, 
    home.name AS HomeTeamName,
    away.name AS AwayTeamName,
    CASE 
        WHEN g.homeGoals > g.awayGoals THEN 'Home Team Wins'
        WHEN g.homeGoals < g.awayGoals THEN 'Away Team Wins'
        ELSE 'TIE' 
    END AS Winning_Side
FROM games g
INNER JOIN teams home ON g.homeTeamID = home.teamID
INNER JOIN teams away ON g.awayTeamID = away.teamID
WHERE g.leagueID = 1
ORDER BY g.date DESC;

 * sqlite:///football_database.db


,season,date,HomeTeamName,AwayTeamName,Winning_Side
0,2020,2021-05-23 15:00:00,Arsenal,Brighton,Home Team Wins
1,2020,2021-05-23 15:00:00,Aston Villa,Chelsea,Home Team Wins
2,2020,2021-05-23 15:00:00,Fulham,Newcastle United,Away Team Wins
3,2020,2021-05-23 15:00:00,Leeds,West Bromwich Albion,Home Team Wins
4,2020,2021-05-23 15:00:00,Leicester,Tottenham,Away Team Wins
...,...,...,...,...,...
2655,2014,2014-08-16 15:00:00,Queens Park Rangers,Hull,Away Team Wins
2656,2014,2014-08-16 15:00:00,Stoke,Aston Villa,Away Team Wins
2657,2014,2014-08-16 15:00:00,West Bromwich Albion,Sunderland,TIE
2658,2014,2014-08-16 15:00:00,West Ham,Tottenham,Away Team Wins


In [140]:
%%sql
-- Calculate the sum of goals scored by each team in the Premier League 2020 and the cumulative sum of goals scored
SELECT home.name AS Home_Team, away.name AS Away_Team, g.homeGoals as Home_Goals, g.awayGoals as Away_Goals, (g.homeGoals + g.awayGoals) AS Total_Goals,
       SUM(g.homeGoals + g.awayGoals) OVER (ORDER BY g.date) AS Cumulative_Goals
FROM games g
LEFT JOIN teams home ON home.teamID = g.homeTeamID
LEFT JOIN teams away ON away.teamID = g.awayTeamID
WHERE g.leagueID = 1 AND g.season = '2020'



 * sqlite:///football_database.db


,Home_Team,Away_Team,Home_Goals,Away_Goals,Total_Goals,Cumulative_Goals
0,Fulham,Arsenal,0,3,3,3
1,Crystal Palace,Southampton,1,0,1,4
2,Liverpool,Leeds,4,3,7,11
3,West Ham,Newcastle United,0,2,2,13
4,West Bromwich Albion,Leicester,0,3,3,16
...,...,...,...,...,...,...
375,Liverpool,Crystal Palace,2,0,2,1024
376,Manchester City,Everton,5,0,5,1024
377,Sheffield United,Burnley,1,0,1,1024
378,West Ham,Southampton,3,0,3,1024


In [137]:
%%sql
/*Calculate the most goals scored by a team in the Premier league 2020 indicating the team name,
the team's side, the opponent name, the date and the maximum goals*/
WITH Max_Goals_Table AS (
    SELECT MAX(homeGoals) AS Max_Goals, homeTeamID AS Team_ID, 'Home' AS Side, date, awayTeamID AS Opponent_ID
    FROM games g
    WHERE g.leagueID =1 AND g.season = '2020'

   UNION ALL

   SELECT MAX(awayGoals) AS Max_Goals, awayTeamID AS Team_ID, 'Away' AS Side, date, homeTeamID AS Opponent_ID
   FROM games g
   WHERE g.leagueID =1 AND g.season = '2020')

SELECT mg.Max_Goals AS 'Maximum Goals in a Game',
       (CASE WHEN mg.Side = 'Home' THEN home.name
       WHEN mg.Side = 'Away' THEN away.name END) AS Team_Name,
       mg.Side, 
       (CASE WHEN mg.Side = 'Home' THEN (SELECT away.name FROM teams away WHERE away.teamID = mg.Opponent_ID)
       WHEN mg.side = 'Away' THEN (SELECT home.name FROM teams home WHERE home.teamID = mg.Opponent_ID) END) AS Opponent_Name,
       strftime('%d-%m-%Y', mg.date) AS Date
FROM Max_Goals_Table mg
LEFT JOIN teams home ON home.teamID = mg.team_ID
LEFT JOIN teams away ON away.teamID = mg.team_ID
WHERE mg.Max_Goals = (SELECT MAX(Max_Goals) FROM Max_Goals_Table)


 * sqlite:///football_database.db


,Maximum Goals in a Game,Team_Name,Side,Opponent_Name,Date
0,9,Manchester United,Home,Southampton,02-02-2021


In [141]:
%%sql
-- Calculate the moving average of goals scored per game for the past 5 games in the 2020 Premier League season
SELECT gameID AS Game_Number,(HomeGoals + AwayGoals) AS Total_Goals, AVG(HomeGoals + AwayGoals) OVER (ORDER BY date ASC ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS "5_Game_Moving_Average_Goals"
FROM games 
WHERE leagueID = 1 AND season = '2020'


 * sqlite:///football_database.db


,Game_Number,Total_Goals,5_Game_Moving_Average_Goals
0,14086,3,3.000000
1,14087,1,2.000000
2,14090,7,3.666667
3,14091,2,3.250000
4,14092,3,3.200000
...,...,...,...
375,14810,2,3.166667
376,14811,5,3.666667
377,14812,1,3.333333
378,14813,3,3.500000


In [142]:
%%sql
-- Find the top 10 players with the highest average goals scored per game
SELECT p.name AS Player_Name, ROUND(AVG(app.goals),2) AS Average_Goals, SUM(goals) AS Total_Goals
FROM appearances app
LEFT JOIN players p ON p.playerID = app.playerID
GROUP BY p.playerID 
HAVING SUM(app.goals) IS NOT NULL and SUM(app.goals) > 10
ORDER BY Average_Goals DESC
LIMIT 10;


 * sqlite:///football_database.db


,Player_Name,Average_Goals,Total_Goals
0,Cristiano Ronaldo,0.96,215
1,Lionel Messi,0.95,231
2,Erling Haaland,0.93,40
3,Robert Lewandowski,0.93,203
4,Zlatan Ibrahimovic,0.79,99
5,Luis Suárez,0.75,168
6,Kylian Mbappe-Lottin,0.72,107
7,Harry Kane,0.71,163
8,Sergio Agüero,0.70,132
9,Neymar,0.68,114


In [143]:
%%sql 
-- Calculate the distribution of players across quintiles based on their average goals scored per game
WITH Quintile_Table AS (SELECT p.name AS Player_Name, ROUND(AVG(app.goals),2) AS Average_Goals, SUM(goals) AS Total_Goals, 
       NTILE(5) OVER (ORDER BY AVG(app.goals) DESC) AS Quintile
FROM appearances app
LEFT JOIN players p ON p.playerID = app.playerID
-- Group by playerID as names could potentially have duplicates and grouping by primary key is usually better
GROUP BY p.playerID 
HAVING SUM(app.goals) IS NOT NULL and SUM(app.goals) > 10
ORDER BY Average_Goals DESC)

SELECT Quintile, COUNT(*) AS Number_of_Players, ROUND(AVG(Average_Goals),2) AS Average_Goals 
FROM Quintile_Table
GROUP BY Quintile
ORDER BY Quintile;


 * sqlite:///football_database.db


,Quintile,Number_of_Players,Average_Goals
0,1,171,0.40
1,2,170,0.24
2,3,170,0.18
3,4,170,0.13
4,5,170,0.08


In [ ]:
%%sql
-- Calculate the top 3 scores in a league over time

In [144]:
%%sql
-- create a query to find Haaland 
SELECT *
FROM players
WHERE name LIKE '%Haaland%';


 * sqlite:///football_database.db


,playerID,name
0,8260,Erling Haaland


In [145]:
#Rank teams according to the number of games won over all seasons for premier league teams

#first query is the previous one with a CTE

#Second Query calculate the HOme_games one for each team

In [146]:
%%sql
-- Rank the teams according to the number of games won over 2014 to 2020 seasons for premier league teams in the dataset 
WITH premMatches AS (SELECT  g.season, 
        g.date, 
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.homeTeamID
        ) AS Home_Team,
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.awayTeamID
        ) AS Away_Team, 
        (CASE WHEN homeGoals > awayGoals THEN "Home Team Wins"
        WHEN homeGoals < awayGoals THEN "Away Team Wins"
        ELSE 'Tie' END) As Winning_Side
FROM games g
WHERE leagueID = 1 
ORDER BY date DESC)

SELECT TEAM, SUM(Wins) AS "Total Wins", RANK () OVER (ORDER BY SUM(Wins) DESC) AS "Rank"
FROM (

SELECT Home_Team AS Team, COUNT(*) As Wins
FROM premMatches 
WHERE Winning_Side = "Home Team Wins"
GROUP BY Home_Team

UNION ALL

SELECT Away_Team AS TEAM, COUNT(*) As Wins
FROM premMatches 
WHERE Winning_Side = "Away Team Wins"
GROUP BY Away_Team
) AS Combined_Wins
GROUP BY TEAM
ORDER BY "Total Wins" DESC
LIMIT 10;


 * sqlite:///football_database.db


,Team,Total Wins,Rank
0,Manchester City,183,1
1,Liverpool,159,2
2,Chelsea,149,3
3,Tottenham,144,4
4,Manchester United,140,5
5,Arsenal,137,6
6,Leicester,111,7
7,Everton,98,8
8,West Ham,94,9
9,Southampton,91,10


In [110]:
%%sql 
-- Calculate the points for each team in the Premier League 2020
SELECT l.name AS League,
       g.season,
       t.name AS Team,
       COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
       --Tie
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
WHERE l.name = 'Premier League' AND g.season = '2020'
GROUP BY l.leagueID, g.season, t.name 
ORDER BY Points DESC


 * sqlite:///football_database.db


,League,season,Team,Points
0,Premier League,2019,Liverpool,99
1,Premier League,2019,Manchester City,81
2,Premier League,2019,Chelsea,66
3,Premier League,2019,Manchester United,66
4,Premier League,2019,Leicester,62
5,Premier League,2019,Tottenham,59
6,Premier League,2019,Wolverhampton Wanderers,59
7,Premier League,2019,Arsenal,56
8,Premier League,2019,Burnley,54
9,Premier League,2019,Sheffield United,54


In [219]:
%%sql 
--Display premier league winning teams showing reiging champions

WITH Season_Points AS (
    SELECT season, t.name AS Team_Name,
        SUM((CASE 
            WHEN g.hometeamID = t.teamID AND g.homeGoals > g.awayGoals THEN 3 
            WHEN g.awayteamID = t.teamID AND g.homeGoals < g.awayGoals THEN 3 END)) + 
        SUM((CASE 
            WHEN g.homeGoals = g.awayGoals THEN 1 END)) AS Season_Points
FROM games g
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
WHERE g.leagueID = 1
GROUP BY t.teamID, season
ORDER BY Season_Points DESC), 

Season_Winners AS (
    -- Identify the winner for each season 
    SELECT season, Team_Name, 
        ROW_NUMBER() OVER(PARTITION BY season ORDER BY Season_Points DESC) AS Season_Rank
    FROM Season_Points 
    GROUP BY season
)

SELECT  
    sw.season, 
    sw.Team_Name AS Champion, 
    LAG(sw.Team_Name, 1) OVER(ORDER BY season) AS Previous_Champion,
    (CASE WHEN sw.Team_Name = (LAG(sw.Team_Name, 1) OVER(ORDER BY season)) THEN 'Yes' ELSE 'No' END) AS Two_Time_Champion
FROM Season_Winners sw
WHERE sw.Season_Rank = 1 
GROUP BY sw.season
ORDER BY sw.season ASC;

 * sqlite:///football_database.db


,season,Champion,Previous_Champion,Two_Time_Champion
0,2014,Chelsea,None,No
1,2015,Leicester,Chelsea,No
2,2016,Chelsea,Leicester,No
3,2017,Manchester City,Chelsea,No
4,2018,Manchester City,Manchester City,Yes
5,2019,Liverpool,Manchester City,No
6,2020,Manchester City,Liverpool,No


In [117]:
%%sql 
-- Analyzes league competitiveness by calculating average points gap between top/bottom teams
-- Lower points gap indicates a more competitive league

WITH season_points AS (SELECT l.name AS League,
       g.season,
       t.name AS Team,
       COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
       --Tie
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY l.leagueID, g.season, t.name 
ORDER BY Points DESC)

SELECT league, 
       ROUND(AVG(Points_GAP),2) AS Avg_Points_Gap,  -- Average gap across seasons
       ROW_NUMBER() OVER (ORDER BY AVG(Points_Gap) DESC) AS Rank
FROM(
     SELECT League, Season, Max(Points) - Min(Points) AS Points_Gap -- Gap per season
     FROM season_points
     GROUP BY League, Season) Sub
GROUP BY League
ORDER BY Avg_Points_Gap DESC

 * sqlite:///football_database.db


,League,Avg_Points_Gap,Rank
0,Premier League,68.86,1
1,Serie A,68.57,2
2,La Liga,64.57,3
3,Ligue 1,63.00,4
4,Bundesliga,59.00,5


In [188]:
%%sql 
-- Analyze league competitiveness by comparing average points and points gaps between top and bottom teams across leagues
WITH season_points AS (SELECT l.name AS League,
       g.season,
       t.name AS Team,
       COUNT((CASE 
            --Home teams wins
            WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
            --Away teams wins
            WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
       --Tie
       COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
FROM games g 
LEFT JOIN leagues l ON l.leagueID = g.leagueID
LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
GROUP BY l.leagueID, g.season, t.name),

league_stats AS (
    SELECT
        league, 
        season,
        MAX(points) - MIN(points) AS Points_Gap_Top_to_Bottom,
        ROUND(AVG(points),2) AS Avg_Points,
        AVG(CASE WHEN Standing <= 4 THEN points END) AS Top_4_Teams_Avg_Points,
        AVG(CASE WHEN Standing >= (SELECT COUNT(*) - 3 FROM season_points sp2 WHERE sp2.league = season_standings.league AND sp2.season = season_standings.season) THEN points END) AS Bottom_4_Teams_Avg_Points
    FROM (
        SELECT 
            league,
            season, 
            team,
            points,
            ROW_NUMBER() OVER (PARTITION BY League, season ORDER BY points DESC) as Standing
        FROM season_points
    ) AS Season_Standings
    GROUP BY league, season
)

SELECT 
    league,
    ROUND(AVG(Avg_Points),2) AS Avg_Points,
    ROUND(AVG(Points_Gap_Top_to_Bottom),2) AS Avg_Points_Gap_Top_to_Bottom,
    ROUND(AVG(Top_4_Teams_Avg_Points - Bottom_4_Teams_Avg_Points),2) AS Points_Gap_Between_Top_and_Bottom_4_Teams    
FROM league_stats
GROUP BY league
ORDER BY Avg_Points DESC;


 * sqlite:///football_database.db


,league,Avg_Points,Avg_Points_Gap_Top_to_Bottom,Points_Gap_Between_Top_and_Bottom_4_Teams
0,Premier League,52.51,68.86,47.75
1,Serie A,52.23,68.57,49.96
2,La Liga,52.13,64.57,48.61
3,Ligue 1,50.09,63.00,42.04
4,Bundesliga,46.78,59.00,38.04


In [ ]:
%%sql
-- use another way of assessing competitiveness of the league such as different winner analysis

In [168]:
#process the data using pandas, When using Jupyter notebooks with SQL magic, we need to separate the SQL and Python code into different cells

stats_df = pd.read_sql_query("""
WITH season_points AS (
    SELECT l.name AS League,
           g.season,
           t.name AS Team,
           COUNT((CASE 
                WHEN homeGoals > awayGoals AND g.homeTeamID = t.teamID THEN 1 
                WHEN homeGoals < awayGoals AND g.awayTeamID = t.teamID THEN 1 END)) * 3 +
           COUNT(CASE WHEN homeGoals = awayGoals THEN 1 END) AS Points 
    FROM games g 
    LEFT JOIN leagues l ON l.leagueID = g.leagueID
    LEFT JOIN teams t ON t.teamID IN (g.homeTeamID, g.awayTeamID)
    GROUP BY l.leagueID, g.season, t.name 
    ORDER BY Points DESC)
SELECT * FROM season_points;
""", conn)

# Group by League and season, calculate stats
stats = stats_df.groupby(['League', 'season'])['Points'].agg([
    ('Total_Points', 'sum'),
    ('Avg_Points', 'mean'),
    ('Std_Points', 'std')
]).round(2)

# For each League and season, get min and max points
points_range = stats_df.groupby(['League', 'season'])['Points'].agg(['min', 'max'])
stats['Points_Gap'] = points_range['max'] - points_range['min']

# Sort by average points
stats = stats.sort_values('Avg_Points', ascending=False)

# Display results
display(stats)

Total_Points  Avg_Points  Std_Points  Points_Gap
League         season                                                  
Premier League 2018            1069       53.45       21.01          82
Serie A        2016            1060       53.00       20.80          76
               2017            1057       52.85       20.55          74
Premier League 2020            1057       52.85       16.89          63
               2016            1056       52.80       19.85          69
...                             ...         ...         ...         ...
Bundesliga     2016             844       46.89       13.94          57
               2020             837       46.50       15.11          62
               2014             836       46.44       13.59          48
               2017             835       46.39       13.99          62
Ligue 1        2019             767       38.35       11.81          55

[35 rows x 4 columns]

In [104]:

%%sql
--New query to test out the pushing to github
SELECT *
FROM games
LIMIT 10;


 * sqlite:///football_database.db


gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeProbability,drawProbability,awayProbability,homeGoalsHalfTime,awayGoalsHalfTime,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,PSCH,PSCD,PSCA
81,1,2015,2015-08-08 15:45:00,89,82,1,0,0.2843,0.3999,0.3158,1,0,1.65,4.0,6.0,1.65,4.0,5.5,1.65,3.6,5.1,1.65,4.09,5.9,1.62,3.6,6.0,1.67,4.0,5.75,1.64,4.07,6.04
82,1,2015,2015-08-08 18:00:00,73,71,0,1,0.3574,0.35,0.2926,0,0,2.0,3.6,4.0,2.0,3.3,3.7,2.1,3.3,3.3,1.95,3.65,4.27,1.91,3.5,4.0,2.0,3.5,4.2,1.82,3.88,4.7
83,1,2015,2015-08-08 18:00:00,72,90,2,2,0.2988,0.4337,0.2675,0,1,1.7,3.9,5.5,1.7,3.5,5.0,1.7,3.6,4.7,1.7,3.95,5.62,1.73,3.5,5.0,1.73,3.9,5.4,1.75,3.76,5.44
84,1,2015,2015-08-08 18:00:00,75,77,4,2,0.6422,0.2057,0.1521,3,0,1.95,3.5,4.33,2.0,3.3,3.75,2.0,3.3,3.6,1.99,3.48,4.34,2.0,3.1,2.7,2.0,3.4,4.33,1.79,3.74,5.1
85,1,2015,2015-08-08 18:00:00,79,78,1,3,0.1461,0.2159,0.638,0,1,2.55,3.3,3.0,2.6,3.2,2.7,2.4,3.2,2.85,2.52,3.35,3.08,2.6,3.1,2.88,2.6,3.25,3.0,2.46,3.39,3.14
86,1,2015,2015-08-08 20:30:00,80,84,2,2,0.0304,0.0924,0.8772,2,1,1.36,5.0,11.0,1.4,4.75,9.0,1.33,4.8,8.3,1.39,4.92,10.39,1.4,4.0,10.0,1.4,5.0,9.5,1.37,5.04,10.88
87,1,2015,2015-08-09 16:30:00,86,74,2,2,0.4386,0.3165,0.2449,1,1,2.88,3.3,2.7,2.8,3.1,2.75,2.65,3.3,2.5,2.88,3.33,2.69,2.7,3.1,2.7,2.88,3.25,2.7,3.09,3.28,2.55
88,1,2015,2015-08-09 16:30:00,83,81,0,2,0.5751,0.2826,0.1423,0,1,1.29,6.0,12.0,1.28,5.75,10.5,1.33,4.8,8.3,1.31,5.75,12.0,1.3,5.0,11.0,1.3,5.75,12.0,1.24,6.75,15.0
89,1,2015,2015-08-09 19:00:00,85,87,0,1,0.2482,0.5454,0.2064,0,0,3.4,3.4,2.3,3.2,3.4,2.3,2.9,3.3,2.3,3.48,3.46,2.25,3.3,3.1,2.3,3.4,3.4,2.3,3.89,3.51,2.09
90,1,2015,2015-08-10 23:00:00,76,88,0,3,0.0615,0.1698,0.7687,0,2,5.75,4.0,1.67,4.75,4.0,1.65,5.1,3.6,1.65,5.75,3.98,1.68,5.5,3.5,1.7,5.5,4.0,1.7,6.46,4.08,1.61


In [ ]:
#automate queries across all tables
for table_name in csv_files.keys():
    query = f"SELECT COUNT(*) AS row_count FROM {table_name};"
    result = pd.read_sql_query(query, conn)
    print(f"Table {table_name} has {result['row_count'][0]} rows.")

In [ ]:
conn = sqlite3.connect('multi_table_database.db')
# Your tables are already saved if using this connection
conn.close()
